In [1]:
import gammalib
import ctools
from cscripts import csobsdef
import csv



In [8]:
name = 'Crab'
id = "001" 
ra = 83.6331
dec = 22.0145
duration = 30
emin = 0.05
emax = 20.0
rad = 8.0

with open('obs_test.csv', 'w') as csvfile:
    fieldnames = ['name', 'id','ra','dec','duration','emin', 'emax', 'rad']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    
#name,id,ra,dec,duration,emin, emax, rad
#080916C,001,83.6331,22.0145,25,0.02,1.0,6.0





In [7]:
!more obs_test.csv
#test = csobsdef()
#test["ra"] = 15

In [ ]:
a = cscripts.csobsdef()
a.inpnt = a['inpnt'].filename('dati.txt')
a.outfile = a['outobs'].filename('outputbuffo.xml')
a.get_parameters()
a.run()
a.execute()